# Welcome to the Schoborg Fly Brain Volume Estimation Jupyter Notebook!
#### By Samuel Fay and Holden Bindl


## Layout

This notebook provides background for the project and explains how volume is estimated for a fly brain. Then, I will go into the code structure while explaining which strategies worked and what I wasn't able to get working.

## Background

Todd Schoborg is a professor in the Department of Molecular Biology at the University of Wyoming. He uses the fruit fly, Drosophila melanogaster, as a model organism to study particular genes involved in the control of brain growth. Mutations of these genes result in a small brain phenotype referred to as microcephaly. One useful metric in measuring the size of a fly's brain is its volume, which can be determined from micro-CT scans of the fly's head.

The micro-CT scan of a fly’s head results in a stack of roughly 150 images that are taken at equal intervals while moving through the fly's head. Each pixel in each image represents a certain volume within that fly's head. The length and width of each pixel are the surface area on an image, and the depth of the pixel is equal to the gap between the images. Throughout his research, Todd has drawn thousands of hand-traced masks, in which each pixel is "on" if there is brain tissue in the pixel and "off" if there is not. Here are a few examples of slices from a single fly head, each of which contains brain tissue, and their corresponding hand-traced masks.

<img src='Images/Brain Scan Example 1.png' width="200" height="200">  <img src='Images/Brain Scan Example 2.png' width="200" height="200">  <img src='Images/Brain Scan Example 3.png' width="200" height="200">

<img src='Images/Hand Drawn Mask 1.png' width="200" height="200">  <img src='Images/Hand Drawn Mask 2.png' width="200" height="200">  <img src='Images/Hand Drawn Mask 3.png' width="200" height="200">

These three images have about 20 images in between each of them, so the changes between each consecutive image are relatively gradual. This allows us to come up with a very good approximation of volume from a collection of masks. By taking the total count of pixels in the mask that are "on" (i.e., that correspond to brain tissue) and then multiplying that by the dimensions of each pixel, we can get a number that represents the volume of the brain in that slice. Then, by doing this for every slice that contains brain tissue and summing the volumes together, we can calculate the total brain volume for the fly. The image below demonstrates taking the volume of a slice that makes up a larger volume. There can be some small errors because each slice (represented in brown in the figure)  is extended for $\Delta x$ without any change in width and height, but if there are enough slices, this method provides a very good approximation.

<img src='Images/Volume Cross Section.png' width="400" height="400">source: math24.net

The main goal of this project is to automate this whole volume calculation process so that Todd does not have to devote a ton of time to manually tracing brain regions in more than 100 slices per fly. To automate this process, we used an image segmentation neural network that classifies each image in a scan as “brain”, or “not brain”. The network was trained using all of the flies that Todd drew masks for. The scans that were shown above are shown below, with the masks that were generated by the neural network.

<img src='Images/Brain Scan Example 1.png' width="200" height="200">  <img src='Images/Brain Scan Example 2.png' width="200" height="200">  <img src='Images/Brain Scan Example 3.png' width="200" height="200">

<img src='Images/Generated Mask 1.png' width="200" height="200">  <img src='Images/Generated Mask 2.png' width="200" height="200">  <img src='Images/Generated Mask 3.png' width="200" height="200">




The scans are not perfect replicates of Todd’s hand-drawn masks, but they are good enough to get a pretty good approximation. They are precise, but not too accurate. The network tends to underestimate the brain region as compared to the hand-drawn scans, but it underestimates it at a pretty consistent rate. Plotting both sets of data resulted in a regression line with an $R^2$ value of $\approx 0.97$. When the network used the training data, the average proportion of the generated volume relative to the actual volume was $\approx 0.87$. If we multiply each predicted volume by $\frac{1}{0.87}\approx 1.15$, there is an average error of about $2.6\%$. If this level of underestimation for the training data is consistent with the results from the new data, then we could get good estimates by multiplying each predicted volume by $1.15$. However, this is a big if, and I don't know what exactly is causing the underapproximation. All I can really say is that the masks the network produces for flies it has not seen before look accurate, so that leads me to think that volume estimations would be precise at the very least.


## Code Documentation

### Code Layout

There are 4 files for this project. In this notebook, I will go over each file in some depth and explain the functions and how they work. For more line by line explanations, the files will have comments in the code.

The general workflow for training the network and then predicting a set of images goes as follows:

1. Read in tif stack training images and convert them to square pngs. Determine if they are a mask or a scan, and then accordingly output them into a genreal Masks/ or Scans/ folder. 

2. Train the network using the Masks/ and Scans/ folder as data. Then, save the model to a file. If the model has already been trained and saved, then you can just change a variable in the code and load in the model.

3. Read in the images you want to predict. Convert them to a square image for the neural network, while storing the original resolution for later rescaling. Predict the images.

4. Take the predicted images and scale them back to the original size for each image. Then, sum the total amount of pixels in each fly and multiply by the voxel size for the image.

### ImageManip.py

ImageManip.py has functions that are used to convert various types of images into a format that can either be used to train the network, or to have the network predict the brain mask. As a little background on how the training images are stored, they are within a parent folder that has 16 fly types folders in it. One fly type name could be "FL Del NLS1-3". Each of these fly type folders has around 5 individual fly folders in it. These individual folders are numbered with an underscore before the number, e.g. "\_02". Within the individual folder, there are two tiff stacks. One of these has the scans, and one has the brain masks. Each has the same number of images in it, which are all in order. Some of the other images given were used in slightly different formats, which there are different functions for. I will explain those when I get to those functions. 

#### writePNG(outDir, flyType, flyNumber, fileName, image, imageNumber, isMaskVar)

This function writes a png to an output directory composed of the input parameters. `outDir` refers to a directory which will have all of the additional specifiers after it: "outDir/flyType/flyNumber". This doesn't incorporate whether it's a mask or not, it was just an early version of the function for testing purposes. I kept it because it maybe could be useful for some testing.

#### writePNGGrouped(outDir, flyType, flyNumber, fileName, image, imageNumber, imagePairID, isMaskVar)

This function is used to write the png in all cases. For this function, `outDir` needs to end in the `flyNumber`, e.g. "someParentDir/flyType/flyNumber". It is this way because in the function that calls this one, I pass in a list of all of the dirs that I want to output images to. If `isMaskVar` is true, then "Masks/" is appended to the end of `outDir`, otherwise "Scans/" is appended. 

Then, the `outPath` variable is made based on some of the arguments:

```python
outPath += (str(imagePairID) + "-" + flyType + "-" + str(flyNumber) + "-" + 
                str(imageNumber) + ".png")
```

`imagePairID` is a number that is unique to that image in whatever directory it is located. `imageNumber` is a number that is unique relative to whatever fly that image is in. So, if the 113th image from fly "\_01" of type "Asp C" is going into a the masks training folder that has 20000 other images before it, then the file name might be "20001-Asp C-\_01-113.png". However, if it is going into a folder where the only images are from Asp C \_01, then the file name would just be "113-Asp C-\_01-113.png". After making the filename, the image is casted to uint16, encoded to a png, and written to file.

#### makeImageSquare(img)

This function crops the longer side of the image evenly from each side so that the image ends up square. If an image is 324x303, then the resulting image will be 303x303. 

```python
if diff % 2 == 0:
    # uneccessary int to satisfy slicing int guarantee
    firstCrop = secondCrop = int(diff / 2)
else:
    firstCrop = int(diff / 2)
    secondCrop = firstCrop + 1
```

If the difference between the longer side and the shorter side is even, then we can chop the same number of pixels off of each side. However, if it isn't, then cropping evenly from each side won't result in an evenly square image. Therefore, we add one pixel to the secondCrop to account for that. After that, we return a sliced copy of the image, just in case the user doesn't want the original image array to be modified. In our case though, when we call this function, we immediately assign the return value to the original input value so it doesn't matter. 

#### noiseReduce(img)

This function just turns each pixel to 0 if the pixel's value is below a certain threshold. In theory, this should make the borders a little more crisp, but it didn't help accuracy when I tried it out just a tiny bit. I didn't even train the network on noise reduced images when I tried it, so that could be why it didn't help. 

It also contains some code from when Holden was working on the project which detects the edges on the image and then makes a square image out of it. One problem with this edge detection code is that it crops it to a fixed size, which could result in cropping out the brain if it's a high resolution image.

#### isMask(tifPath)

This function determines if a tif stack is consists of masks or scans. Masks only have two unique values throughout all of the images, while scans are grayscale, with values between 0 and 65535 if it's read in as a uint16. This means that if there are more than 2 unique values in the image, then it must be a scan. If there are only 2 unique values, then it is most likely a mask. However, I make sure that there are two images that have exactly 2 unique values in them, just in case there's a scan that is mostly black and then has one pixel that is a value other than 0. 

#### getSpecificImageFromTif(tifPath, index, getScan=False)

One of the issues that this network has is underestimating the volume. One solution Lars suggested for this is to replicate some the images with the largest absolute error in the training dataset. One of the things I output when I test the network on the training set is the index of the image with the largest error for each fly. Then, to replicate these images in the training dataset, we need both the mask and the scan from the index. To indicate whether we need the mask or the scan, there is the optional argument `getScan` with a default value of `False`. If the function is supplied the scan tif file, and we need the mask, or is it's supplied the mask file and we need the scan, then we find the other tif file in the current directory. This is assuming that in each folder for each fly follows the same format as the training data, where there are two tif stack files, one with scans and one with masks. If this is the case, then we just iterate through the directory, and once we find a file that is not hidden, and doesn't have the same name as the one we don't want, then we choose that file.

Now that we have the correct tif stack which we want to get, we just iterate through each page and increment a counter varable until we reach the desired index. Then, we return that image. 

#### convertImageHelper(tifPath, outputDirectory, totalImages, imagePairID)

This is a helper function for the different functions that convert tif images to png images for training or testing. It takes in a path to the tif to convert, the output directory, `totalImages`, and `imagePairID`. `totalImages` is used to keep track of how many images each fly has. `imagePairID` is used to give each fly a unique number that can me matched to the corresponding mask or scan. `imagePairID` is not incremented after this function returns if the corresponding folder of the current fly hasn't been processed yet. This logic is done in the one of the parent functions `convertAndNormalizeTestImages()`, but it is still relevant to this function. T

`writePNGGrouped()` is then called, which writes the image using the unique number ID `imagePairID` and a local variable `imageNumber`, which denotes which number the current image is with respect to the current fly. The function then returns totalImages to indicate the total number of images that have been processed in the current file and all the files before it. It also returns a dictionary with a key of the name and type of the fly, and a value of the side dimension of the square image of the fly before it is scaled down to 256x256. This is a necessary value to have, because when calculating the brain volume of an image, the masks must be scaled back up to the original square resolution of the image so that each pixel takes up the area that it was intended to take up.

#### convertAndNormalizeTestImages(parentDirectory, outputDirectory, totalImages, imagePairID):

This function takes in a parent directory of the format `parentDirectory/flyType/flyNumber` where each fly number folder has two tif image stack files in it, one of scans and one of masks. The outputDirectory is a directory that will be used for training, so it just contains two folders of images, one for masks and one for scans. If being done on a new folder without already existing uniquely numbered data, then `totalImages` and `imagePairID` can be initialized to 1. 

All of the fly directories are looped through. For each directory, if it's the first pass through the directory, which means we haven't processed either the scans or masks yet, so the `firstPass` variable is `True`. When first pass is true, we set `previousTotalImages` equal to `totalImages`, and then we run `convertImageHelper()` and return the result from that into `totalImages`. Then we get the difference between `totalImages` and `previousTotalImages` to get how many images the current fly contains, and store this in `firstPassImageCount`. 

For the second pass, we also create `previousTotalImages` in order to make sure that there is no mismatch between the number of masks and scans for every fly. We call the function, get the new `totalImages`, look at the difference and make sure that `numberNewImages` is equalt to `firstPassImageCount`. If it's not, we print a message warning that there is an image mismatch in the current fly. Then we increment `imagePairID` by `numberNewImages` because we have written both the scans and the masks for the current fly.

As an example, lets say `totalImages` starts at 300 and `imagePairID` start at 151, meaning we've written one total fly so far with 150 images, and we are just starting on the second one. on the first pass, we iterate through the fly directory and happen to choose the mask file. We call `convertImageHelper()` and it assigns image IDs 151-305 to the images which are written to the `Masks/` folder in `parentDirectory`, which means that this second fly has 155 images. `convertImageHelper()` then returns total images, which is 455. In the current function, `imagePairID` is still `151` because it isn't returned from `convertImageHelper()`, it's only passed to it. We set `firstPass` to False, continue in the loop, and then process the scan file. We pass `totalImages=455` and `imagePairID=151` to `convertImageHelper()`, which processes the images in the tif scans file. That returns our new value for `totalImages`, which is hopefully `610`. We check that the images that were just processed from the scan file is equal to the images from the mask file, and then we increment `imagePairID` by `numberNewImages`, so `imagePairID` is now `306`.

#### convertAndNormalizeSpecificTestImages(tifDirectoryList, outputDirectory, totalImages, imagePairID):

This function just takes a list of directories of tifs and calls `convertImageHelper()` for them. It doesn't do any unique pair IDs for the images, it just outputs them to the output directory. I only used it once for testing purposes.

#### convertToDifferentDirectories(tifFileList, outputDirectoryList):

This function converts from a the tif file at index `i` in `tifFileList` to the directory at index `i` in `outputDirectoryList`. It also returns a dictionary of the original image resolutions, which can be used later if you are predicting and need to rescale the predicted masks back to the resolution they are supposed to be at for accurate volume calculations. This is the function that I used to convert the training data from tifs to pngs when I was testing how the network predicted on the training data.

### SEMANTIC2.py

SEMANTIC2.py contains the code for the image segmentation neural network. Much of the code for the network comes straight from the Tensorflow Image Segmentation tutorial, which can be found [here](https://www.tensorflow.org/tutorials/images/segmentation). The architecture of the network remains the same apart from the resolution that the images are processed at, and what is changed is mainly how the images are loaded in, how the training data is processed, and the model being saved.


`BUFFER_SIZE = 1000
BATCH_SIZE = 32
IMG_WIDTH = 224
IMG_HEIGHT = 224
\# brain, or not brain
OUTPUT_CHANNELS = 2`

Buffer size and batch size are just set to values that worked for me. They definitely could be changed. The width and heights of the images are set to 224 because that is the max dimensions that the unet model this network is made with accepts. The original tutorial dataset had pictures of pets with masks associated with them. The masks had 3 types of pixels, one for pet, one for the border of pet, and one for not pet. The training masks only have 2 types of pixels, one for brain and one for not brain. When I first started to train the network, the output channels was still at 3 and it worked totally fine, so it may not be super necessary to change it down to 2, but it probably doesn't hurt.